# Laboratório Exame
## Equipe:
## Pedro Anacleto Martins Senna De Oliveira
## André Luiz de Melo Thissen
## Nikollas da Silva Antes
## Pedro Luchiari de Carvalho

## Importação de bibliotecas

In [611]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from keras.models import Sequential
from keras.layers import Dense
from sklearn.ensemble import RandomForestRegressor

## Importando DataSets
- Datasets foram unidos para que o Label Encoding fosse o mesmo para categorias iguais, uma vez que o dataset de treinamento tinha mais categoria.

In [612]:
teste = pd.read_csv('exame_cmc13_dados_teste.csv',sep=';',on_bad_lines='skip')
treinamento = pd.read_csv('exame_cmc13_dados_treinamento.csv',sep=';',on_bad_lines='skip')
total = pd.concat([treinamento,teste])

- Verificação de que há elementos nulos com posterior remoção desses elementos.

In [613]:
print(total.isnull().values.any())
total = total.dropna(how='any')
print(total.isnull().values.any())

True
False


- Verificação da porcentagem de dados de teste e treinamento a fim da separação do dataset.

In [614]:
print(treinamento.index)
print(teste.index)

RangeIndex(start=0, stop=131179, step=1)
RangeIndex(start=0, stop=32795, step=1)


- Remoção de colunas que foram tidas como irrelevantes para a classificação dos filmes.

In [615]:
total = total.drop(columns='Unnamed: 0.1')
total = total.drop(columns='Unnamed: 0')
total = total.drop(columns='user_id')
total = total.drop(columns='isbn')
total = total.drop(columns='year_of_publication')
total= total.drop(columns='img_l')
total= total.drop(columns='Language')
total = total.drop(columns='Category')
total= total.drop(columns='city')
total = total.drop(columns='state')
total.loc[:,'age'] =total.loc[:,'age'].astype(int)
total = total [['country','publisher','age','book_title','book_author','rating']]

- Separação dos dados em atributos independentes e dependente.

In [616]:
variables =pd.DataFrame(total.iloc[:,0:5].values)
results = total.iloc[:,5].values

- Label Encoding

In [617]:
variables.loc[:, 0] =  LabelEncoder().fit_transform(variables.iloc[:, 0])
variables.loc[:,1] = LabelEncoder().fit_transform((variables.iloc[:,1]))
variables.loc[:,3] = LabelEncoder().fit_transform((variables.iloc[:,3]))
variables.loc[:,4] = LabelEncoder().fit_transform((variables.iloc[:,4]))
variables

,0,1,2,3,4
0,172,31,34,839,205
1,172,31,34,839,205
2,172,31,35,839,205
3,172,31,29,839,205
4,172,31,31,839,205
...,...,...,...,...,...
156456,172,74,59,257,181
156457,172,96,34,798,22
156458,172,31,25,795,167
156459,172,5,55,418,13


- Split em dados de treinamento e teste

In [618]:
variables_treinamento, variable_test, results_treinamento, results_teste = train_test_split(variables, results, test_size = 0.25, random_state = 0)
print('Formato dos dados de treinamento: ',variables_treinamento.shape)
print('Formato dos dados de teste: ',variable_test.shape)
print('Formato dos dados de treinamento: ',results_treinamento.shape)
print('Formato dos dados de teste: ',results_teste.shape)

Formato dos dados de treinamento:  (117345, 5)
Formato dos dados de teste:  (39116, 5)
Formato dos dados de treinamento:  (117345,)
Formato dos dados de teste:  (39116,)


- Normalização dos dados

In [619]:
sc = StandardScaler()
variables_treinamento = sc.fit_transform(variables_treinamento)
variable_test = sc.transform(variable_test)

## Model Evaluation - Redes Neurais do Tipo MLP

In [620]:
classificador = Sequential()
classificador.add(Dense(2, kernel_initializer = 'uniform', activation = 'relu', input_dim = 5))
classificador.add(Dense( 3, kernel_initializer = 'uniform', activation = 'relu'))
classificador.add(Dense( 11, kernel_initializer = 'uniform', activation = 'softmax'))
classificador.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
classificador.fit(variables_treinamento, results_treinamento, batch_size = 100, epochs= 20)

Epoch 1/20
1174/1174 [==============================] - 2s 1ms/step - loss: 1.6500 - accuracy: 0.5935
Epoch 2/20
1174/1174 [==============================] - 1s 975us/step - loss: 1.4342 - accuracy: 0.5938
Epoch 3/20
1174/1174 [==============================] - 1s 1ms/step - loss: 1.4329 - accuracy: 0.5938
Epoch 4/20
1174/1174 [==============================] - 1s 991us/step - loss: 1.4328 - accuracy: 0.5938
Epoch 5/20
1174/1174 [==============================] - 1s 1ms/step - loss: 1.4328 - accuracy: 0.5938
Epoch 6/20
1174/1174 [==============================] - 1s 974us/step - loss: 1.4327 - accuracy: 0.5938
Epoch 7/20
1174/1174 [==============================] - 1s 1ms/step - loss: 1.4326 - accuracy: 0.5938
Epoch 8/20
1174/1174 [==============================] - 1s 1ms/step - loss: 1.4328 - accuracy: 0.5938
Epoch 9/20
1174/1174 [==============================] - 1s 1ms/step - loss: 1.4327 - accuracy: 0.5938
Epoch 10/20
1174/1174 [==============================] - 1s 1ms/step - loss:

In [621]:
resultsPred = classificador.predict(variable_test)
resultsPred

1223/1223 [==============================] - 1s 714us/step


array([[0.5873554 , 0.00157073, 0.00274408, ..., 0.10587692, 0.07205503,
        0.0797283 ],
       [0.6004832 , 0.00137717, 0.00244187, ..., 0.10381611, 0.06995583,
        0.07734184],
       [0.5763359 , 0.0017504 , 0.00302064, ..., 0.10754726, 0.07379507,
        0.08170936],
       ...,
       [0.61464226, 0.00119116, 0.00214665, ..., 0.10150845, 0.06766169,
        0.07473799],
       [0.61155564, 0.00122981, 0.00220843, ..., 0.10201897, 0.06816437,
        0.07530817],
       [0.5917018 , 0.00150427, 0.00264084, ..., 0.10520307, 0.07136309,
        0.07894125]], dtype=float32)

## Análise dos Resultados

In [622]:
vetorPred = np.zeros((39116,1)).astype(int)
cm = confusion_matrix(results_teste, vetorPred)
print(cm)
accuracy_score(results_teste,vetorPred)

[[23189     0     0     0     0     0     0     0     0     0     0]
 [   60     0     0     0     0     0     0     0     0     0     0]
 [  102     0     0     0     0     0     0     0     0     0     0]
 [  203     0     0     0     0     0     0     0     0     0     0]
 [  291     0     0     0     0     0     0     0     0     0     0]
 [ 1420     0     0     0     0     0     0     0     0     0     0]
 [ 1196     0     0     0     0     0     0     0     0     0     0]
 [ 2706     0     0     0     0     0     0     0     0     0     0]
 [ 4095     0     0     0     0     0     0     0     0     0     0]
 [ 2872     0     0     0     0     0     0     0     0     0     0]
 [ 2982     0     0     0     0     0     0     0     0     0     0]]


0.5928264648737089

## Model Evaluation - Random Forest

In [633]:
rf = RandomForestRegressor(n_estimators = 20)
rf.fit(variables_treinamento, results_treinamento)
rfPred = rf.predict(variable_test)
rfPred = rfPred.astype(int)
rfPred

array([4, 2, 3, ..., 3, 5, 1])

## Análise dos Resultados

In [635]:
cm = confusion_matrix(results_teste, rfPred)
print(cm)
accuracy_score(results_teste,rfPred)

[[4120 3256 4287 4711 3083 1709 1094  557  283   81    8]
 [  19   11    7    5   10    3    2    3    0    0    0]
 [  24   18   18   18   14    3    2    1    4    0    0]
 [  26   25   36   53   35   12    8    4    4    0    0]
 [  49   36   59   74   31   24    4   10    4    0    0]
 [ 197  181  271  347  189  116   65   34   18    2    0]
 [ 168  160  240  249  177   96   50   29   23    4    0]
 [ 340  354  504  627  387  240  136   70   43    5    0]
 [ 513  469  767  930  627  346  236  131   58   18    0]
 [ 385  326  456  581  501  285  165  104   46   22    1]
 [ 352  296  451  607  481  332  210  138   71   40    4]]


0.11639738214541365

## Conclusão
 - Percebe-se que as redes neurais tiveram um desempenho melhor uma vez que tiveram a taxa de acerto maior. Outro fator importante a ser mencionado, trata-se que a rede neurai, nesse exemplo, foi enviezada de tal forma que todas as predições dela tendiam a zero, o que é fator devido aos dados fornecidos.A rede neurai possui uma diferença no fato de que dão probalidade para cada tipo de saída com base nos inputs, trazendo uma precisão maior e uma taxa de aprendizagem melhor.